# **0. Before Reading**

# **1. Introduction**
이상치 탐지의 비지도 학습은 기계학습을 기반으로 하고 있으며 보안이나 헬스케어 등 많은 분야에서 활용되고 있습니다. 이상치 탐지의 핵심은 밀도 추정(densitiy estimation); 많은 입력 샘플이 주어졌을 때, 이상치는 낮은 확률로 density area에 존재합니다.
지난 몇 년간 많은 진전이 있었음에도 다차원/고차원 데이터에서의 이상치 탐지의 비지도 학습은 도전과제로 남아있습니다. 특히 입력 데이터의 차원이 증가할수록 original feature space에서 입력 샘플이 관측될 확률이 낮아 밀도 추정이 어려워집니다. 차원 수에 의한 이슈를 다루기 위해 차원 축소를 실행하고 나서 낮은 차원의 공간에서 밀도 추정이 실행됩니다. 그러나 이러한 접근은 먼저 실행된 차원 축소 과정에서 차후의 밀도 추정에서 반영되지 않거나 핵심 정보가 삭제되어 최적의 이상치 탐지 성능을 내지 못할 수 있습니다. 그러므로 차원 축소와 밀도 추정의 영향을 합치는 것이 최적화를 위해 계산하는 것이 힘들더라도 성능을 위해서 수행되어야 합니다. 최근 여러 연구에서 deep network 모델 활용을 연구했지만, 그 결과는 핵심 정보가 보전되지 않은 차원 축소된 낮은 차원 공간 모델보다 성능이 제한되었습니다.

fig 1

이 논문에서는 앞에 언급한 문제를 다룰 수 있는 비지도 이상치 탐지 모델인 Deep Autoencoding Gaussian Mixture Model(DAGMM)을 제안합니다. 
먼저 DAGMM은 축소된 차원과 복원 오차의 정보(reconstruction error)가 포함한 입력 샘플의 핵심 정보를 낮은 차원 공간으로 전달합니다. fig1을 보면 이상치와 정상치의 다른 점을 볼 수 있습니다. (1) 이상치는 축소된 차원에서 관련된 특징과는 다른 방식으로 크게 벗어나있습니다. (2) 이상치는 정상치와 비교했을 때 복원하기 어렵습니다. DAGMM은 기존의 메소드와는 달리 compression network라고 불리는 sub-network를 차원 축소에 활용합니다. 이 autoencoder는 인코더에서 차원 축소된 특징과 디코더에서 복원 오차를 결합해 낮은 차원에서의 정보를 준비합니다.
두 번째로 DAGMM은 낮은 차원에서 복잡한 구조의 데이터에 대한 밀도 추정을 실행
q하기 위해 Gaussian Mixture Model(GMM)을 도입했습니다. GMM의 성능이 강력하더라도 모델 학습에 도입하는 것은 어려운 문제입니다. GMM은 Expectation-Maximization과 같은 대체 알고리즘으로 학습되는데 종종 기존 방식의 접근법으로 변질하여서 차원 축소와 밀도 추정을 최적화하는 것이 어렵습니다. DAGMM은 이 문제를 처리하기 위해 estimation network라는 sub-network를 활용합니다. estimation network는 compression network의 입력과 각 샘플의 예측값을 입력으로 사용합니다. 예측된 샘플을 통해 GMM의 파라미터를 추정하여 입력의 energy/likelihood 검증이 용이합니다. compression network에서의 복원 오차와 estimation network에서의 sample energy를 일제히 최소화함으로써, 목표한 밀도 추정에 도움이 되는 차원 축소 정보를 학습할 수 있습니다.
마지막으로 DAGMM은 end-to-end 학습에 적합합니다. 보통 deep autoencoder는 덜 최적화된 상태에 고착되기 쉬워 end-to-end 방식으로 학습되기 어려워 pre-training 방식을 사용하기도 합니다. 그러나 fine-tuning을 통해 잘 학습된 autoencoder를 만들기 어려워 차원 축소를 조정하는 잠재적 성능을 제한합니다. DAGMM은 정규화를 통해 estimation network가 compression network 내부의 autoencoder가 고착화된 상태에서 벗어나는 것을 도와 end-to-end 방식으로 잘 학습됩니다.

# **2. Related Work**
생략

# **3. Deep Autoencoding Gaussian Mixture Model**
## **3.1 Overview**
fig 2
DAGMM은 크게 compression network와 estimation network로 이루어져 있습니다. 그림2와 같이 (1) compression network는 deep autoencoder를 통해 입력 샘플의 차원 축소를 실행합니다. 그리고 축소된 공간과 복원 오차로 낮은 차원의 정보를 estimation network로 연결합니다. (2) estimation network에서는 입력된 정보로 GMM로 likelihood/energy를 예측합니다.

## **3.2 Compression Netwok**
낮은 차원의 정보는 compression network의 두 소스: (1) deep autoencoder를 통해 학습된 축소된 낮은 차원의 정보와 (2) 복원 오차에서 파생된 특징을 통해 준비됩니다. 샘플 x가 주어졌을 때, compression network는 입력의 낮은 차원 정보 z를 계산합니다.
$$\begin{matrix} 
&\mathbf z_c = h(\mathbf x;\theta_e),&& x = g(\mathbf z_c;\theta_d) &(1)\\ 
&&\mathbf z_r = f(\mathbf x,\mathbf x’) &&(2)&\\
&&\mathbf z = [\mathbf z_c,\mathbf z_r] &&(3)&\\ 
\end{matrix}$$

$\mathbf z_c$는 축deep autoencoder를 통해 학습된 축소된 낮은 차원의 정보  
$\mathbf z_r$은  복원 오차에서 파생된 특징  
$\theta_e$와 $\theta_d$는 deep autoencoder의 파라미터  
$\mathbf x’$은 복원된 $\mathbf x$  
$h(⋅)$는 인코딩 함수  
$g(⋅)$는 디코딩 함수  
$f(⋅)$는 복원 오차를 계산하는 함수  
특히 $\mathbf z_r$은 다양한 거리 측정법을 생각하면 다차원이 될 수 있습니다.   
마지막으로 compression network는 $\mathbf z$를 estimation network에 연결합니다.  

## **3.3 Estimation Network**
학습 단계에서 알 수 없는 혼합 분포 $\phi$, 평균 $\mu$, 공분산 $\Sigma$, estimation network의 GMM의 파라미터를 추정하고 EM을 사용하지 않고 샘플의 likelihood/energy를 검증합니다. estimation network는 다층신경망을 활용하여 각 샘플의 혼합 구성을 예측하는데 성공합니다. 낮은 차원의 정보 $\mathbf z$와 혼합 구성 요소의 수 $K$가 주어졌을 때, estimation network는 다음과 같이 예측합니다.

$$\mathbf p = MLN(\mathbf z;\theta_m), \qquad \hat{\gamma} = \mathbf{softmax}(\mathbf p) \qquad (4)$$

$\hat{\gamma}$는 매끄러운 혼합 멤버쉽 예측을 위한 K차원 벡터이고 p는 다층신경망$\theta_m$의 출력이다. N개의 샘플의 batch와 예측값이 주어졌을 때, $\forall1 \leq k \leq K$, GMM의 파라미터를 구할 수 있습니다.
$$\hat\phi_k = \sum^N_{i=1} \frac{\hat\gamma_{ik}} {N}, \qquad \hat\mu_k = \frac{\sum^N_{i=1} \hat\gamma_{ik}\mathbf z_i} {\sum^N_{i=1} \hat\gamma_{ik}}, \qquad \hat\Sigma_k = \frac{\sum^N_{i=1} \hat\gamma_{ik} (\mathbf z_i-\hat\mu_k)(\mathbf z_i-\hat\mu_k)^T}{\sum^N_{i=1} \hat\gamma_{ik}} \qquad (5)$$ 

$\hat{\gamma}_i$는 낮은 차원 정보 $\mathbf z_i$에서 구성 요소 예측이고 GMM의 k번 째 구성 요소에서 $\hat{\phi}_k, \hat{\mu}_k, \hat{\Sigma}_k$는 각각 확률, 평균, 공분산입니다.

$$ E(\mathbf z) = -\log \left( \sum^K_{k=1} \hat\phi_k \frac {\exp \left( -\frac {1} {2}(\mathbf z-\hat\mu_k)^T \hat{\Sigma}^{-1}_k(\mathbf z-\hat\mu_k) \right)} {\sqrt{|2\pi \hat\Sigma_k}|} \right) $$

$|\cdot|$은 행렬식

GMM 파라미터를 검증하는 단계에서 샘플 에너지를 추정하는 것과 사전에 선택된 threshold를 통해 높은 에너지를 가진 샘플을 이상치로 예측하는 것은 간단합니다.

## **3.4 Objective Function**
샘플의 수가 N개인 데이터가 주어졌을 때, DAGMM을 학습시키는 기능 함수는 다음과 같습니다.

$$J(\theta_e, \theta_d, \theta_m) = \frac {1} {N} \sum^N_{i=1} L(\mathbf x_i ,\mathbf x_i ) + \frac {\lambda_1} {N} \sum^N_{i=1} E(\mathbf z_i) + \lambda_2 P(\hat\Sigma).$$ (7)

$L(\mathbf x_i, \mathbf x’_i)$는 복원 오차를 나타내는 loss 함수입니다. 직관적으로 compression network가 낮은 복원 오차을 낮게 생성할 수 있다면, 낮은 차원의 표현은 좀 더 입력 샘플의 중요 정보를 보존할 수 있게 될 것입니다. 그러므로 낮은 차원의 복원 오차의 compression network는 항상 요구됩니다. 실제로 L_2 정규화는 좀 더 좋은 결과를 나타냅니다. $L(\mathbf x_i, \mathbf x’_i) = || \mathbf x_i - \mathbf x'_i ||^2_2$

$E(\mathbf z_i)$ 모델은 입력 샘플이 발견될 확률입니다. 샘플 에너지를 최소화함으로써 likelihood가 최대가 되는 최선의 compression과 estimation network의 결합을 찾습니다.

DAGMM은 GMM에서 공분산 행렬의 대각 성분이 모두 0으로 소실 될 때 자명해가 발생하는 특이점 문제를 갖고 있습니다. 이 문제를 피하고자 대각 성분에 작은 값을 부여합니다.
$P(\hat\Sigma) = \sum^N_{k=1} \sum^d_{j=1} \frac {1} {\hat{\Sigma}_{kij}} $ d는 낮은 차원 표현의 수
lambda1과 lambda2는 DAGMM의 메타 파라미터입니다. $\lambda1 = 0.1, \lambda2 = 0.005$로 주로 사용합니다.

## **3.5 Relation to Variational Inference**
DAGMM에서 각 샘플의 멤버쉽 예측을 위해 estimation network를 도입했습니다. fig1을 보면 estimation network가 잠재 변수를 예측과 유사한 역할을 합니다. 최근 어려운 잠재 변수 추론 문제를 다루기 위해 깊은 신경망을 난해하고 확장될 수 없는 기존의 접근법인 정밀한 모델 추론에 적용하는 것이 제안됐다. 이론적으로 DAGMM의 멤버쉽 예측을 신경 변수 예측에 적용할 수 있습니다. 샘플 xi의 경우, 에너지 함수에 대한 압축된 표현인 z_i 기여는 다음과 같이 상한 될 수 있습니다.
$$\begin{align} 
E(\mathbf z_i) = −\log \sum_k p(\mathbf z_i)=\ &−\log \sum_k p(\mathbf z_i, k)\\ 
=\ &−\log \sum_k Q_{\theta_m}(k\ |\ \mathbf z_i) \frac {p(\mathbf z_i, k)}  {Q_{\theta_m}(k\ |\ \mathbf z_i)}\\
≤\ & −\sum_k Q\theta_m(k\ |\ \mathbf z_i) \log \frac {p(\mathbf z_i, k)} {Q_{\theta_m}(k\ |\ \mathbf z_i)}\\
=\ & −E_{Q_{\theta_m}} [\log p(\mathbf z_i, k) − \log Q_{\theta_m}(k\ |\ \mathbf z_i)] &(8)\\
=\ & −E_{Q_{\theta_m}} [\log p(\mathbf z_i\ |\ k)] + KL(Q_{\theta_m}(k\ |\ \mathbf z_i)||p(k)) &(9)\\
=\ & − \log p(\mathbf z_i) + KL(Q_{\theta_m}(k\ |\ \mathbf z_i)||p(k\ |\ \mathbf z_i))\\
=\ & E(\mathbf z_i) + KL(Q_{\theta_m}(k\ |\ \mathbf z_i)||p(k\ |\ \mathbf z_i)) &(10)& \end{align}$$

$Q_{\theta_m}(k | \mathbf z_i)$는 $z_i$의 멤버쉽을 예측하는 estimation network이고, $KL(·||·)$은 두 입력 분산에서 Kullback-Leibler 발산이고, $p(k) = \phi_k$는 추정될 혼합 계수이고, $p(k | \mathbf z_i)$는 혼합 구성 분포의 조건부 사후 확률 분포입니다. 방정식(8)에서 음의 성분을 최소화함으로써 true posterior에 가까워지고 에너지 함수의 범위를 견고하게 할 수 있습니다. DAGMM에서 에너지 함수가 평가하는데 다루기 쉽고 효율적이기 때문에 방정식(10) 대신 (6)을 목적 함수의 한 부분으로 사용합니다. 위에서 설명한 사후 분포를 정의하기 위한 deep estimation network를 사용한 신경 변수 추론과 달리 DAGMM은 뚜렷하게 표본 의존적 사후 분포를 파라미터화 하기 위해 deep estimation network를 사용합니다. 이전의 기계학습 연구를 보면, mixture of experts같은 혼합 모델에서 신경망을 활용하여 샘플 멤버쉽을 계산한 연구가 있습니다. 이 관점에서, DAGMM은 deep autoencoder가 결합된 mixture of experts의 비지도학습 버전이라고 볼 수 있습니다. 

## **3.6 Training Strategy**
사전학습 기반의 deep autoencoder와 달리, DAGMM은 end-to-end 학습을 사용했습니다. 먼저 이 논문에서는 사전학습된 compression network가 이상치 탐지의 성능 저하를 일으키는 것을 발견했습니다. 두 번째로 compression network와 estimation network가 서로의 성능을 높여 줄 수 있다는 것을 발견했습니다. 한편 estimation network를 통한 정규화와 end-to-end 방식으로 학습된 deep autoencoder는 pre-trained 된 것과 유사하게 복원 오차를 줄일 수 있는 반면, deep autoencoder 단독으로 end-to-end 학습시킨 것으로는 같은 성능을 달성할 수 없습니다. 다른 한편 잘 학습된 compression network로부터의 낮은 차원 표현과 함께, estimation network는 유의미한 밀도 추정이 가능합니다.

# **4. Experimental Results**
## **4.1 Dataset**
UCI repository의 KDDCUP은 41차원의 샘플을 갖고있습니다. 34개의 차원은 연속형 나머지 7개는 범주형입니다. 범주형 피쳐를 one-hot encoding하여 120차원의 얻습니다. 데이터 샘플의 20%는 “normal” 나머지는 “attack”으로 라벨링 되어, “normal” 샘플이 소수 그룹이 되므로 이상치로 처리하여 이상치 탐지를 진행합니다.

## **4.2 Baseline Methods**
**기존의 SOTA methods**
- OC_SVM(One-Class Support Vector Machine)
- DSEBM-e(Deep Structured Energy Based Model): 에너지로 이상치 판단
- DSEBM-r: 복원 오차로 이상치 판단
- DCN(Deep Clustering Network): k-means 기반의 autoencoder. 군집에서 멀어질수록 이상치로 판단

**DAGMM 기반**
- GMM-EN: DAGMM의 목적 함수에서 복원 오차 삭제, 샘플 에너지로 이상치 탐지
- PAE: DAGMM의 목적 함수에서 에너지 함수 삭제, deep autoencoder와 동일, 복원 오차로 이상치 판단
- E2E-AE: PAE에서 autoencoder를 end-to-end 방식으로 학습
- PAE-GMM-EM: 두 단계 접근법을 적용. 첫 단계에서 compression network를 deep autoencoder를 사전학습시킴으로써 compression network를 학습. 다음 단계에서 estimation network의 GMM을 EM 알고리즘으로 학습. 샘플 에너지로 이상치 탐지
- PAE-GMM: PAE-GMM의 두 번째 단계에서 compression network를 estimation network로 학습
- DAGMM-p: DAGMM과 PAE-GMM을 절충하여 compression network를 사전학습하고 end-to-end로 DAGMM을 미세 조정. 셈플 에너지로 이상치 탐지
- DAGMM-NVI: DAGMM과의 차이는 신경 변수 추론을 채택한 것과 방정식(6)을 (10)으로 대체하는 것입니다.

## **4.3 DAGMM Configuration**
모든 실험에서 compression network의 복원 특징을 relative Euclidean distance와 cosine similarity로 사용합니다. 샘플 $\mathbf x$와 복원 $\mathbf x$’이 주어질 때, relative Euclidean dsitance는 $ \frac {\|\mathbf x-\mathbf x’\|_2} {\|\mathbf x\|_2}$ 이고 cosine similarity는 $\frac {\mathbf x \cdot \mathbf x’} {\|\mathbf x\|_2\|\mathbf x’\|_2}$

**DAGMM의 network 구조**
estimation network는 compression network로부터 축소된 차원에서 하나, 복원 오차에서 둘 총 3차원 입력을 받습니다. estimation network는 GMM의 4개의 혼합 구성을 검토합니다.
the compression network runs with FC(120, 60, tanh)-FC(60, 30, tanh)-FC(30, 10, tanh)-FC(10, 1, none)-FC(1, 10, tanh)-FC(10, 30, tanh)-FC(30, 60, tanh)-FC(60, 120, none), and the estimation network performs with FC(3, 10, tanh)-Drop(0.5)-FC(10, 4, softmax).

**FC(a, b,c)**: fully-connected layer with a input, b output, f activation fucntion)  
**Drop(p)**: dropout layer with p probability  
trained by **Adam**  
**learning rate** 0.0001  
**epochs** 200  
**mini-batches** 1024  
**lambda1** 0.1  
**lambda2** 0.005  

## **4.4 Accuracy**
**metric**
이상치 탐지의 성능을 비교하기 위해 precision, recall, F1 score의 평균을 고려했습니다. table1의 이상치 피율을 기반으로 이상치 샘플의 threshold를 정했습니다. KDDCUP의 경우 에너지의 샘플의 상위 20%를 이상치로 정했습니다.
table 2는 20회 실행 후 precision, recall, F1 score의 평균을 보여줍니다. 일반적으로 DAGMM 모든 데이터 셋에서 가장 높은 F1 점수를 얻었습니다. 특히 KDDCUP과 KDDCUP-Rev 데이터셋에서 기존 메소드와 비교했을 때 F1 점수가 14%, 10% 향상 되었습니다.
OC-SVM에서는 차원 수가 성능 저하의 주요한 원인이 될 수 있습니다.
DSEBM에서는 여러 데이터 셋에서 잘 작동하지만 DAGMM이 에너지 모델링에서 낮은 차원 표현과 복원 오차를 함께 고려하기 때문에 성능이 우수합니다.
DCN, PAE-GMM, DAGMM-p의 성능은 사전학습된 deep autoencoder로 인해 제한 될 수 있습니다. 잘 학습된 deep autoencoder에서 축소된 차원과 밀도 추정 문제에서 유의미한 변화를 만들어내기 어렵습니다.
GMM-EN은 복원 오차 없이는 밀도 추정이 어려워 보입니다.
PAE의 경우 복원 오차의 한쪽 면만으로는 이상치 탐지에 충분하지 않을 수 있습니다.
E2E-AE는 PAE와 DAGMM만큼 복원 오차를 줄일 수 없습니다.
샘플의 중요한 정보가 차원 축소가 면서 소실되기 때문에 E2E-AE는 KDDCUP과 Thyroid에서 낮은 성능을 보여줍니다. 
DAGMM과 DAGMM-NVI의 성능은 유사합니다
GMM은 단순 그래픽 모델이기 때문에 DAGMM의 신경 변수 추론을 통한 성능 개선을 발견하지 못했습니다.



refer
https://angeloyeo.github.io/2021/02/08/GMM_and_EM.html